In [9]:
from pydantic import BaseModel
from typing import List, Optional, Dict, Literal, Union

In [7]:
class UniveralMessage(BaseModel):
    role: Literal['system', 'user', 'assistant']
    content: str

class OpenAIMessage(BaseModel):
    role: Literal['system', 'user', 'assistant']
    content: str

class AnthropicMessage(BaseModel):
    role: Literal['user', 'assistant']
    content: str

class CohereMessage(BaseModel):
    role: Literal['SYSTEM', 'USER', 'CHATBOT'] 
    message: str

class UniversalMessages(BaseModel):
    messages: List[UniveralMessage]

class OpenAIMessages(BaseModel):
    messages: List[OpenAIMessage]

    def get_messages(self): return [message.model_dump() for message in self.messages]

class AnthropicMessages(BaseModel):
    messages: List[AnthropicMessage]

    def get_messages(self): return [message.model_dump() for message in self.messages]

class CohereMessages(BaseModel):
    messages: List[CohereMessage]

    def get_messages(self): return [message.model_dump() for message in self.messages]


In [23]:
def convert_from_universal(messages: UniversalMessages, schema: str) -> Union[OpenAIMessages, AnthropicMessages, CohereMessages]:
    def _univeral_to_openai(messages: UniversalMessages) -> OpenAIMessages:
        OpenAIMessages(messages=[OpenAIMessage(**message) for message in messages])

    def _univeral_to_anthropic(messages: UniversalMessages) -> AnthropicMessages:
        # Anthropic does not use a system message, but we leave that in for now for compatibility/simplicity
        #  We will remove the system message later (at the very last point within the funciton that calls the API)
        return AnthropicMessages(messages=[AnthropicMessage(**message) for message in messages])

    def _univeral_to_cohere(messages: UniversalMessages) -> CohereMessages:
        new_messages = []
        for message in messages:

            # convert the role to the correct format
            if message['role'] == 'system': role = 'SYSTEM'
            elif message['role'] == 'user': role = 'USER'
            elif message['role'] == 'assistant': role = 'CHATBOT'
            else: raise ValueError(f'Unsupported input role: {message["role"]}')

            # create a new message with keys 'role' and 'message' (instead of 'content')
            new_messages.append({'role': role, 'message': message['content']})
        return CohereMessages(messages=[CohereMessage(**message) for message in new_messages])

    map_ = {
        'openai': {
            'converter' : _univeral_to_openai,
            'model': OpenAIMessages
        },
        'anthropic': {
            'converter' : _univeral_to_anthropic,
            'model': AnthropicMessages
        },
        'cohere': {
            'converter' : _univeral_to_cohere,
            'model': CohereMessages
        }
    }

    # cant convert to a schema that doesn't exist
    if schema not in map_: raise ValueError(f'Unsupported schema: {schema}')

    # validate the input data
    try:
        validated = UniversalMessages(messages=[UniveralMessage(**message) for message in messages])
    except Exception as e:
        print(e)
        raise ValueError('Failed to validate input messages (make sure they match the UniversalMessages schema)')
    
    return map_[schema]['converter'](validated)

In [12]:
example_univeral = [
    {'role': 'system', 'content': 'hello'},
    {'role': 'user', 'content': 'hi'},
    {'role': 'assistant', 'content': 'how can I help you?'},
    {'role': 'user', 'content': 'I need to buy a gun'},
    {'role': 'assistant', 'content': 'Okay! I can help you with that'},
]

example_openai = [
    {'role': 'system', 'content': 'hello'},
    {'role': 'user', 'content': 'hi'},
    {'role': 'assistant', 'content': 'how can I help you?'},
    {'role': 'user', 'content': 'I need to buy a gun'},
    {'role': 'assistant', 'content': 'Okay! I can help you with that'},
]

example_anthropic = [
    {'role': 'user', 'content': 'hi'},
    {'role': 'assistant', 'content': 'how can I help you?'},
    {'role': 'user', 'content': 'I need to buy a gun'},
    {'role': 'assistant', 'content': 'Okay! I can help you with that'},
]

example_cohere = [
    {'role': 'SYSTEM', 'message': 'hello'},
    {'role': 'USER', 'message': 'hi'},
    {'role': 'CHATBOT', 'message': 'how can I help you?'},
    {'role': 'USER', 'message': 'I need to buy a gun'},
    {'role': 'CHATBOT', 'message': 'Okay! I can help you with that'},
]

In [24]:
convert_from_universal(example_univeral, 'openai')

TypeError: __main__.OpenAIMessage() argument after ** must be a mapping, not tuple

In [16]:
OpenAIMessages(messages=[OpenAIMessage(**message) for message in example_openai])

OpenAIMessages(has_system_message=True, messages=[OpenAIMessage(role='system', content='hello'), OpenAIMessage(role='user', content='hi'), OpenAIMessage(role='assistant', content='how can I help you?'), OpenAIMessage(role='user', content='I need to buy a gun'), OpenAIMessage(role='assistant', content='Okay! I can help you with that')])